Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Первая задача — проанализировать базу данных. Полученные данные помогут сформулировать ценностное предложение для нового продукта.





## Поставленные задачи:
1.  Посчитайте, сколько книг вышло после 1 января 2000 года;

2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

3. Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;

4. Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;

5. Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'здесь был пароль', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Задача №1

Посчитайте, сколько книг вышло после 1 января 2000 года?

In [3]:
query = '''SELECT count(book_id)
FROM books
where publication_date > '2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


## Задача №2

Для каждой книги посчитайте количество обзоров и среднюю оценку.

tab1.*, tab2.average_rating
FROM tab1
left join tab2 using(book_id)
order by average_rating desc

In [8]:
query = '''
with tab1 as (select b.book_id, b.title, count(re.review_id) number_of_reviews
from books b
left join reviews re using(book_id)
group by b.book_id),

tab2 as (select b.book_id, avg(ra.rating) average_rating
from books b
left join ratings ra using(book_id)
group by b.book_id)


SELECT tab1.*, tab2.average_rating
FROM tab1
left join tab2 using(book_id)
order by number_of_reviews desc
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,number_of_reviews,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


## Задача №3

Определите издательство, которое выпустило наибольшее число книг толще
50 страниц

In [5]:
query = '''
with tab1 as (select publisher_id, count(book_id) count
from books
where num_pages > 50
group by publisher_id
order by count desc
limit 1)


SELECT publisher_id, publisher
FROM publishers
where publisher_id in (select publisher_id from tab1)
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,212,Penguin Books


## Задача №4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [6]:
query = '''
with tab1 as (select b.book_id, count(ra.rating_id)
from books b
left join ratings ra using(book_id)
group by b.book_id
having count(ra.rating_id) >= 50)

SELECT a.author_id, a.author,  avg(ra.rating) average_rating
FROM books b
left join ratings ra using(book_id)
left join authors a using(author_id)
where b.book_id in (select book_id from tab1)
group by a.author_id
order by average_rating desc
limit 1


'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


## Задача №5

Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

In [7]:
query = '''
with tab1 as (SELECT username
FROM ratings
group by username
having count(username) > 48),

tab2 as (select username, count(review_id)
from reviews
where username in (select * from tab1)
group by username)

select avg(count)
from tab2
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


## **Результаты исследования:**



1. После 1 января 2000 года вышло 819 книг.

2. Для каждой книги посчитано количество обзоров и среднюю оценку.
Можно заметить, что большое количество обзоров не гарантирует высокого рейтинга.

3. Издательство, которое выпустило наибольшее число книг толще
50 страниц - Penguin Books

4. Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. Перспективно будет посмотреть, по авторам имеющимся в продаже культовым сериям произведений, кто же обладает более лояльной и заинтересованной аудиторией и учитывать этот фактор в составлении предлагаемой линейки книг в приложении.

5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора. Рекомендуем расмотреть создания системы кураторов/экспертов, которые могут делать обзоры, тематические подборки и активнее вовлекать пользователей в использования приложений.